In [57]:
import pandas as pd
import numpy as np
import cx_Oracle
import os
import json
import datetime
import warnings

warnings.filterwarnings(action="ignore")
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
import bs4
import requests
import time
import math
import random

pd.set_option('display.max_columns', None)

In [82]:
df = pd.read_csv("busan_arriving_vessels_new_final_1.csv")
df

,vessel_name,tos_vessel_name,mmsi,imo,eta,destination,navigation_info,longitude,latitude,heading,speed,max_draught,time_position_data_received,time_voyage_data_received,CAPACITY,tos_ETA,tos_ETD,VOYAGE,DIS_QTY,LOAD_QTY,time_difference,isnottoday,timestamp,remained_distance,estimated_fuel_consumption,predicted_ETA
0,ZHUCHENGXINZHOU,ZHU CHENG XIN ZHOU,413556780,9252993.0,9231200,JPOSA,under way using engine,127.4252,33.9390,85.0,12.4,8.0,2022-09-20 04:50:36.000000,2022-09-17 10:51:37.000000,2524,2022-09-21 00:00:00.000000,2022-09-21 15:00:00.000000,MZXZ031,500,500,2 days 17:58:59,0 days 09:04:55.449643,2022-09-20 13:55:31.449643,171.679039,221.755791,2022-09-20 12:18:36
1,MSC KANOKO,MSC KANOKO,636019214,9842102.0,9200600,KRPUS,under way using engine,125.4184,32.9038,207.0,11.5,13.0,2022-09-19 21:59:10.000000,2022-09-18 12:52:34.000000,14336,2022-09-21 01:00:00.000000,2022-09-22 02:00:00.000000,MKAO005,1199,86,1 days 09:06:36,0 days 15:56:21.449643,2022-09-20 13:55:31.449643,388.813185,1088.582901,2022-09-20 16:14:10
2,OOCL NEW YORK,OOCL NEW YORK,477274700,9198109.0,9200900,KP PUS,under way using engine,127.9110,33.7376,149.0,1.8,10.0,2022-09-20 00:35:34.000000,2022-09-19 03:19:52.000000,5770,2022-09-21 06:00:00.000000,2022-09-21 17:00:00.000000,OONW001,766,241,0 days 21:15:42,0 days 13:19:57.449643,2022-09-20 13:55:31.449643,160.377572,981.103686,2022-09-22 00:41:34
3,MSC BERYL,MSC BERYL,357926000,9467392.0,9200900,BUSAN,under way using engine,125.5706,34.0615,116.0,13.2,14.0,2022-09-20 04:28:09.000000,2022-09-18 02:40:28.000000,12967,2022-09-21 06:00:00.000000,2022-09-22 00:00:00.000000,MBER006,1126,825,2 days 01:47:41,0 days 09:27:22.449643,2022-09-20 13:55:31.449643,314.306545,870.920244,2022-09-20 17:19:09
4,VALIANT,VALIANT,229541000,9628178.0,9200900,CN NBO,under way using engine,128.4482,28.5231,277.0,17.6,10.0,2022-09-20 03:15:38.000000,2022-09-20 01:33:31.000000,8827,2022-09-21 18:00:00.000000,2022-09-22 14:00:00.000000,VLIX007,1200,400,0 days 01:42:07,0 days 10:39:53.449643,2022-09-20 13:55:31.449643,717.676222,2174.527932,2022-09-21 01:16:38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
764328,SEASPAN BREEZE,SEASPAN BREEZE,477127900,9685358.0,12060800,MEXICO,moored,-104.2922,19.0660,10.0,0.0,11.0,2022-12-07 09:42:00.000000,2022-12-07 09:41:58.000000,10100,2022-12-28 18:00:00.000000,2022-12-29 14:00:00.000000,OZRE009,1200,400,0 days 00:00:02,0 days 09:04:45.366027,2022-12-07 18:46:45.366027,11800.225274,NaN,NaN
764329,MSC ELSA3,MSC ELSA 3,636016814,9123221.0,12090330,SINGAPORE,under way using engine,108.3167,8.3983,NaN,14.0,8.0,2022-12-07 07:44:20.000000,2022-12-04 01:34:19.000000,1730,2022-12-29 21:00:00.000000,2022-12-30 14:00:00.000000,MLSA002,200,200,3 days 06:10:01,0 days 11:02:25.366027,2022-12-07 18:46:45.366027,3618.160466,185.719557,2022-12-13 03:16:20
764330,MSC INDIA,MSC INDIA,255806005,9231248.0,10241200,ZHOU SHAN,moored,122.0806,29.7822,214.0,0.0,15.0,2022-12-05 04:33:35.000000,2022-12-05 04:39:47.000000,902,2022-12-30 00:00:00.000000,2022-12-31 01:00:00.000000,MNDA001,500,800,-1 days +23:53:48,2 days 14:13:10.366027,2022-12-07 18:46:45.366027,855.845779,NaN,NaN
764331,OOCL NEW YORK,OOCL NEW YORK,477274700,9198109.0,12070800,INPAV,under way using engine,72.4769,19.2479,353.0,13.2,12.0,2022-12-07 09:34:20.000000,2022-12-07 09:31:50.000000,5770,2022-12-30 16:00:00.000000,2022-12-31 12:00:00.000000,OONW003,600,500,0 days 00:02:30,0 days 09:12:25.366027,2022-12-07 18:46:45.366027,5765.511097,428.436386,2022-12-17 05:24:20


In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 764333 entries, 0 to 764332
Data columns (total 26 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   vessel_name                  764333 non-null  object 
 1   tos_vessel_name              764333 non-null  object 
 2   mmsi                         764333 non-null  int64  
 3   imo                          764331 non-null  float64
 4   eta                          764333 non-null  int64  
 5   destination                  761315 non-null  object 
 6   navigation_info              761490 non-null  object 
 7   longitude                    764333 non-null  float64
 8   latitude                     764333 non-null  float64
 9   heading                      726991 non-null  float64
 10  speed                        764331 non-null  float64
 11  max_draught                  763631 non-null  float64
 12  time_position_data_received  764333 non-null  object 
 13 

In [84]:
df["timestamp"] = pd.to_datetime(df["timestamp"])
df["time_position_data_received"] = pd.to_datetime(df["time_position_data_received"])
df["time_voyage_data_received"] = pd.to_datetime(df["time_voyage_data_received"])
df["tos_ETA"] = pd.to_datetime(df["tos_ETA"])
df["tos_ETD"] = pd.to_datetime(df["tos_ETD"])
df["predicted_ETA"] = pd.to_datetime(df["predicted_ETA"])


In [85]:
import datetime

In [86]:
df = df[df["timestamp"] > datetime.datetime(year=2022, month=11, day=1)]
df

,vessel_name,tos_vessel_name,mmsi,imo,eta,destination,navigation_info,longitude,latitude,heading,speed,max_draught,time_position_data_received,time_voyage_data_received,CAPACITY,tos_ETA,tos_ETD,VOYAGE,DIS_QTY,LOAD_QTY,time_difference,isnottoday,timestamp,remained_distance,estimated_fuel_consumption,predicted_ETA
155774,XIN YA ZHOU,XIN YA ZHOU,413161000,9334935.0,10311100,KRPUS,under way using engine,128.6616,34.0045,322.0,0.7,13.0,2022-10-31 14:57:31,2022-10-31 02:46:51,8176,2022-11-01 08:00:00,2022-11-02 12:00:00,CXYZ003,711,1293,0 days 12:10:40,0 days 09:04:33.629580,2022-11-01 00:02:04.629580,108.285128,2714.578441,2022-11-04 02:28:31
155775,MSC RUBY,MSC RUBY,636092764,9502960.0,11010400,KRPUS,under way using engine,129.1444,31.5316,310.0,21.1,12.0,2022-10-31 14:47:20,2022-10-31 02:16:07,13092,2022-11-01 13:00:00,2022-11-02 12:00:00,MRUB007,1674,222,0 days 12:31:13,0 days 09:14:44.629580,2022-11-01 00:02:04.629580,383.480234,1753.587099,2022-11-01 00:35:20
155776,VALOR,VALOR,229458000,9628154.0,11011000,KR PUS,under way using engine,126.7983,32.9208,72.0,9.4,12.0,2022-10-31 14:59:06,2022-10-31 02:42:43,8800,2022-11-01 19:00:00,2022-11-03 03:00:00,VLOX008,1037,1907,0 days 12:16:23,0 days 09:02:58.629580,2022-11-01 00:02:04.629580,294.364093,713.919678,2022-11-01 07:53:06
155777,ZHUCHENGXINZHOU,ZHU CHENG XIN ZHOU,413556780,9252993.0,11010230,JPHKT,under way using engine,132.8183,32.6749,267.0,15.8,8.0,2022-10-31 14:48:10,2022-10-31 03:00:39,2524,2022-11-02 00:00:00,2022-11-02 15:00:00,MZXZ037,500,500,0 days 11:47:31,0 days 09:13:54.629580,2022-11-01 00:02:04.629580,448.893122,710.854480,2022-11-01 06:08:10
155778,VALUE,VALUE,229467000,9628166.0,11020400,BUSAN KOREA,under way using engine,133.1280,31.9932,240.0,9.1,11.0,2022-10-31 10:30:27,2022-10-31 08:49:42,8800,2022-11-02 13:00:00,2022-11-03 13:00:00,VLUX007,921,400,0 days 01:40:45,0 days 13:31:37.629580,2022-11-01 00:02:04.629580,518.751257,1270.657457,2022-11-01 17:16:27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
764328,SEASPAN BREEZE,SEASPAN BREEZE,477127900,9685358.0,12060800,MEXICO,moored,-104.2922,19.0660,10.0,0.0,11.0,2022-12-07 09:42:00,2022-12-07 09:41:58,10100,2022-12-28 18:00:00,2022-12-29 14:00:00,OZRE009,1200,400,0 days 00:00:02,0 days 09:04:45.366027,2022-12-07 18:46:45.366027,11800.225274,NaN,NaT
764329,MSC ELSA3,MSC ELSA 3,636016814,9123221.0,12090330,SINGAPORE,under way using engine,108.3167,8.3983,NaN,14.0,8.0,2022-12-07 07:44:20,2022-12-04 01:34:19,1730,2022-12-29 21:00:00,2022-12-30 14:00:00,MLSA002,200,200,3 days 06:10:01,0 days 11:02:25.366027,2022-12-07 18:46:45.366027,3618.160466,185.719557,2022-12-13 03:16:20
764330,MSC INDIA,MSC INDIA,255806005,9231248.0,10241200,ZHOU SHAN,moored,122.0806,29.7822,214.0,0.0,15.0,2022-12-05 04:33:35,2022-12-05 04:39:47,902,2022-12-30 00:00:00,2022-12-31 01:00:00,MNDA001,500,800,-1 days +23:53:48,2 days 14:13:10.366027,2022-12-07 18:46:45.366027,855.845779,NaN,NaT
764331,OOCL NEW YORK,OOCL NEW YORK,477274700,9198109.0,12070800,INPAV,under way using engine,72.4769,19.2479,353.0,13.2,12.0,2022-12-07 09:34:20,2022-12-07 09:31:50,5770,2022-12-30 16:00:00,2022-12-31 12:00:00,OONW003,600,500,0 days 00:02:30,0 days 09:12:25.366027,2022-12-07 18:46:45.366027,5765.511097,428.436386,2022-12-17 05:24:20


In [87]:
df[["tos_vessel_name", "longitude","latitude","time_position_data_received"]]

,tos_vessel_name,longitude,latitude,time_position_data_received
155774,XIN YA ZHOU,128.6616,34.0045,2022-10-31 14:57:31
155775,MSC RUBY,129.1444,31.5316,2022-10-31 14:47:20
155776,VALOR,126.7983,32.9208,2022-10-31 14:59:06
155777,ZHU CHENG XIN ZHOU,132.8183,32.6749,2022-10-31 14:48:10
155778,VALUE,133.1280,31.9932,2022-10-31 10:30:27
...,...,...,...,...
764328,SEASPAN BREEZE,-104.2922,19.0660,2022-12-07 09:42:00
764329,MSC ELSA 3,108.3167,8.3983,2022-12-07 07:44:20
764330,MSC INDIA,122.0806,29.7822,2022-12-05 04:33:35
764331,OOCL NEW YORK,72.4769,19.2479,2022-12-07 09:34:20


In [88]:
idx= df[["tos_vessel_name", "longitude","latitude","time_position_data_received"]].drop_duplicates().index

In [89]:
df = df.loc[idx,:]

In [90]:
drop_idx = df[df["destination"].isna()].index
df.drop(drop_idx, inplace=True)

In [91]:
df["destination"] = df["destination"].apply(lambda x: str(x).upper())

In [92]:
df = df.query("destination.str.contains('KR') | destination.str.contains('PUS') | destination.str.contains('BUS') | destination.str.contains('BNP')")

In [93]:
df.reset_index(inplace=True, drop=True)

In [94]:
dindex = df["timestamp"].value_counts().index.sort_values()
dindex

DatetimeIndex(['2022-11-01 00:02:04.629580', '2022-11-01 00:07:06.579991',
               '2022-11-01 00:17:13.992266', '2022-11-01 00:22:15.439804',
               '2022-11-01 00:32:20.970502', '2022-11-01 00:37:23.688904',
               '2022-11-01 00:42:26.336936', '2022-11-01 00:47:29.561893',
               '2022-11-01 00:52:32.801123', '2022-11-01 00:57:35.084567',
               ...
               '2022-12-07 17:56:34.486886', '2022-12-07 18:06:35.870642',
               '2022-12-07 18:11:36.515809', '2022-12-07 18:16:40.025638',
               '2022-12-07 18:21:38.621756', '2022-12-07 18:26:38.936792',
               '2022-12-07 18:31:41.628857', '2022-12-07 18:36:40.712211',
               '2022-12-07 18:41:41.358578', '2022-12-07 18:46:45.366027'],
              dtype='datetime64[ns]', length=9105, freq=None)

In [79]:
df.query(f"timestamp == '{dindex[-10]}'")

,vessel_name,tos_vessel_name,mmsi,imo,eta,destination,navigation_info,longitude,latitude,heading,speed,max_draught,time_position_data_received,time_voyage_data_received,CAPACITY,tos_ETA,tos_ETD,VOYAGE,DIS_QTY,LOAD_QTY,time_difference,isnottoday,timestamp,remained_distance,estimated_fuel_consumption,predicted_ETA
30516,MSC PETRA,MSC PETRA,636014644,9222302.0,12061100,BUSAN,moored,128.8246,35.0773,91.0,0.0,12.0,2022-12-07 06:58:47,2022-12-07 06:32:29,6621,2022-12-07 08:00:00,2022-12-08 09:00:00,MPET001,518,1521,0 days 00:26:18,0 days 09:00:57.044552,2022-12-07 15:59:44.044552,11.984889,NaN,NaT
30517,ONE ORINOCO,ONE ORINOCO,477836900,9937311.0,12071100,KRPUS,under way using engine,129.2163,34.7352,321.0,2.2,13.0,2022-12-07 06:58:53,2022-12-07 06:44:28,0,2022-12-07 20:00:00,2022-12-09 13:00:00,OORI003,1205,2293,0 days 00:14:25,0 days 09:00:51.044552,2022-12-07 15:59:44.044552,44.616957,4.989743,2022-12-07 17:55:53
30518,COSCO FOS,COSCO FOS,477947700,9484302.0,12070800,KR PUS,under way using engine,125.0204,34.5110,244.0,0.9,10.0,2022-12-07 06:59:07,2022-12-05 08:37:09,4253,2022-12-09 02:00:00,2022-12-09 23:00:00,CFOS028,987,600,1 days 22:21:58,0 days 09:00:37.044552,2022-12-07 15:59:44.044552,350.836589,129.655494,2022-12-16 01:28:07
30519,CSCL WINTER,CSCL WINTER,477117400,9645877.0,12071100,KRPUS,under way using engine,128.3745,34.2183,182.0,6.8,14.0,2022-12-07 06:58:55,2022-12-07 06:42:46,10036,2022-12-10 00:00:00,2022-12-10 20:00:00,CSWI003,600,600,0 days 00:16:09,0 days 09:00:49.044552,2022-12-07 15:59:44.044552,92.939980,11.256089,2022-12-07 14:20:55
30520,HEUNG-A ULSAN,HEUNG-A ULSAN,440120000,9128996.0,12080700,KRPUS WA,moored,131.0039,33.9694,292.0,0.0,6.0,2022-12-07 06:56:32,2022-12-07 06:56:01,364,2022-12-10 22:00:00,2022-12-11 08:00:00,HUSN048,80,80,0 days 00:00:31,0 days 09:03:12.044552,2022-12-07 15:59:44.044552,229.056231,NaN,NaT


In [66]:
df["VOYAGE"].value_counts()

MGNI014    2246
MBAI003    1012
MIDA019     675
CFOS027     670
CFOS025     656
           ... 
MNIS001      30
MSOR001      26
MMMM003      21
MNDR007      19
MAEE002       1
Name: VOYAGE, Length: 113, dtype: int64

In [70]:
df_test = df.query("VOYAGE == 'MGNI014'").query("navigation_info != 'moored'").reset_index(drop=True)
df_test

,vessel_name,tos_vessel_name,mmsi,imo,eta,destination,navigation_info,longitude,latitude,heading,speed,max_draught,time_position_data_received,time_voyage_data_received,CAPACITY,tos_ETA,tos_ETD,VOYAGE,DIS_QTY,LOAD_QTY,time_difference,isnottoday,timestamp,remained_distance,estimated_fuel_consumption,predicted_ETA
0,MSC GIANNINA II,MSC GIANNINA II,373579000,9289051.0,11130100,BUSAN NEW PORT,under way using engine,128.8269,35.0773,272.0,0.3,8.0,2022-11-10 22:55:44,2022-11-10 10:53:45,1736,2022-11-18 11:00:00,2022-11-19 08:00:00,MGNI014,100,100,0 days 12:01:59,0 days 09:01:02.729159,2022-11-11 07:56:46.729159,11.994909,250.377611,2022-11-11 20:30:44
1,MSC GIANNINA II,MSC GIANNINA II,373579000,9289051.0,11130100,BUSAN NEW PORT,under way using engine,128.8149,35.0752,257.0,7.6,8.0,2022-11-10 23:05:34,2022-11-10 11:06:24,1736,2022-11-18 11:00:00,2022-11-19 08:00:00,MGNI014,100,100,0 days 11:59:10,0 days 09:01:14.995233,2022-11-11 08:06:48.995233,11.749508,12.402366,2022-11-10 23:55:34
2,MSC GIANNINA II,MSC GIANNINA II,373579000,9289051.0,11130100,BUSAN NEW PORT,under way using engine,128.8008,35.0727,257.0,9.1,8.0,2022-11-10 23:10:45,2022-11-10 11:06:24,1736,2022-11-18 11:00:00,2022-11-19 08:00:00,MGNI014,100,100,0 days 12:04:21,0 days 09:01:07.157248,2022-11-11 08:11:52.157248,11.589324,11.823434,2022-11-10 23:51:45
3,MSC GIANNINA II,MSC GIANNINA II,373579000,9289051.0,11130100,BUSAN NEW PORT,under way using engine,128.7851,35.0635,194.0,9.8,8.0,2022-11-10 23:17:06,2022-11-10 11:12:22,1736,2022-11-18 11:00:00,2022-11-19 08:00:00,MGNI014,100,100,0 days 12:04:44,0 days 09:04:52.033867,2022-11-11 08:21:58.033867,10.899744,11.162437,2022-11-10 23:53:06
4,MSC GIANNINA II,MSC GIANNINA II,373579000,9289051.0,11130100,BUSAN NEW PORT,under way using engine,128.7818,35.0379,175.0,10.0,8.0,2022-11-10 23:26:04,2022-11-10 11:12:22,1736,2022-11-18 11:00:00,2022-11-19 08:00:00,MGNI014,100,100,0 days 12:13:42,0 days 09:00:56.960386,2022-11-11 08:27:00.960386,8.332938,8.559602,2022-11-10 23:52:04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
785,MSC GIANNINA II,MSC GIANNINA II,373579000,9289051.0,11240800,BUSAN NEW PORT,under way using engine,128.7998,35.0052,340.0,13.0,8.0,2022-11-23 22:55:05,2022-11-23 10:47:39,1736,2022-11-24 08:00:00,2022-11-25 01:00:00,MGNI014,411,516,0 days 12:07:26,0 days 09:00:34.796518,2022-11-24 07:55:39.796518,4.344487,5.036933,2022-11-23 23:05:05
786,MSC GIANNINA II,MSC GIANNINA II,373579000,9289051.0,11240800,BUSAN NEW PORT,under way using engine,128.7954,35.0163,339.0,12.7,8.0,2022-11-23 22:58:12,2022-11-23 10:59:39,1736,2022-11-24 08:00:00,2022-11-25 01:00:00,MGNI014,411,516,0 days 11:58:33,0 days 09:02:30.887380,2022-11-24 08:00:42.887380,5.636034,6.424213,2022-11-23 23:12:12
787,MSC GIANNINA II,MSC GIANNINA II,373579000,9289051.0,11240800,BUSAN NEW PORT,under way using engine,128.7845,35.0458,0.0,10.8,8.0,2022-11-23 23:08:02,2022-11-23 11:05:40,1736,2022-11-24 08:00:00,2022-11-25 01:00:00,MGNI014,411,516,0 days 12:02:22,0 days 09:02:46.216744,2022-11-24 08:10:48.216744,9.051256,9.481577,2022-11-23 23:35:02
788,MSC GIANNINA II,MSC GIANNINA II,373579000,9289051.0,11240800,BUSAN NEW PORT,under way using engine,128.7868,35.0617,16.0,9.0,8.0,2022-11-23 23:13:51,2022-11-23 11:05:40,1736,2022-11-24 08:00:00,2022-11-25 01:00:00,MGNI014,411,516,0 days 12:08:11,0 days 09:02:00.226110,2022-11-24 08:15:51.226110,10.663978,10.883531,2022-11-23 23:51:51


In [81]:
df

,vessel_name,tos_vessel_name,mmsi,imo,eta,destination,navigation_info,longitude,latitude,heading,speed,max_draught,time_position_data_received,time_voyage_data_received,CAPACITY,tos_ETA,tos_ETD,VOYAGE,DIS_QTY,LOAD_QTY,time_difference,isnottoday,timestamp,remained_distance,estimated_fuel_consumption,predicted_ETA


In [96]:
cnt = 30
for idx in dindex[30:60]:
    cnt +=1
    df_ = df.query(f"timestamp == '{idx}'")
    df_.to_csv(f"visualization_{cnt}.csv", encoding="utf-8 sig", header=True, index=False)

In [113]:
# 선형적인 방법
def fuel_estimation(teu, T, S):
    T = T / 24
    if teu <= 1500:
        first_co_eff = 0.0072
        sec_co_eff = 10.8592
        estimated_fuel_consumption = (0.0072 * T * math.pow(S, 3)) + (10.8592 *
                                                                      T)

    elif teu <= 3000:
        first_co_eff = (teu - 1500) * 0.00000096 + 0.0072
        sec_co_dff = (teu - 1500) * 0.00312713 + 10.8592
        estimated_fuel_consumption = (first_co_eff * T *
                                      math.pow(S, 3)) + (sec_co_dff * T)

    elif teu <= 4500:
        first_co_eff = (teu - 3000) * 0.00000096 + 0.0072
        sec_co_dff = (teu - 3000) * 0.00312713 + 10.8592
        estimated_fuel_consumption = (first_co_eff * T *
                                      math.pow(S, 3)) + (sec_co_dff * T)

    elif teu <= 5750:
        first_co_eff = (teu - 4500) * 0.00000076 + 0.0101
        sec_co_dff = (teu - 4500) * 0.00467896 + 20.2406
        estimated_fuel_consumption = (first_co_eff * T *
                                      math.pow(S, 3)) + (sec_co_dff * T)

    elif teu <= 7000:
        first_co_eff = (teu - 5750) * 0.00000076 + 0.0101
        sec_co_dff = (teu - 5750) * 0.00467896 + 20.2406
        estimated_fuel_consumption = (first_co_eff * T *
                                      math.pow(S, 3)) + (sec_co_dff * T)

    elif teu <= 8500:
        first_co_eff = (teu - 7000) * 0.0000007 + 0.012
        sec_co_dff = (teu - 7000) * 0.0004734666 + 31.9380
        estimated_fuel_consumption = (first_co_eff * T *
                                      math.pow(S, 3)) + (sec_co_dff * T)

    elif teu <= 10000:
        first_co_eff = (teu - 8500) * 0.0000007 + 0.012
        sec_co_dff = (teu - 8500) * 0.0004734666 + 31.9380
        estimated_fuel_consumption = (first_co_eff * T *
                                      math.pow(S, 3)) + (sec_co_dff * T)

    elif teu <= 11750:
        first_co_eff = (teu - 10000) * 0.000001114 + 0.0141
        sec_co_dff = (teu - 10000) * 0.000027171428 + 32.1584
        estimated_fuel_consumption = (first_co_eff * T *
                                      math.pow(S, 3)) + (sec_co_dff * T)

    elif teu <= 13500:
        first_co_eff = (teu - 11750) * 0.000001114 + 0.0141
        sec_co_dff = (teu - 11750) * 0.000027171428 + 32.1584
        estimated_fuel_consumption = (first_co_eff * T *
                                      math.pow(S, 3)) + (sec_co_dff * T)

    else:
        first_co_eff = 0.0180
        sec_co_dff = 32.2535
        estimated_fuel_consumption = (first_co_eff * T *
                                      math.pow(S, 3)) + (sec_co_dff * T)

    return estimated_fuel_consumption

In [141]:
cosco_df = pd.read_csv("voy_path/COSCO_FOS_VRKR6_26.csv")
cosco_df

,vessel_name,mmsi,imo,CAPACITY,destination,eta,longitude,latitude,speed,heading,timestamp,time_position_data_received,time_voyage_data_received,navigation_info,remained_distance,estimated_fuel_consumption
0,COSCO FOS,477947700,9484302.0,4253,CNTSN,2022-11-17 09:00:00,127.0882,34.1223,16.0,270.0,2022-11-16 16:56:08.041108,2022-11-16 07:55:49,2022-11-16 04:19:07,under way using engine,184.436450,306.205403
1,COSCO FOS,477947700,9484302.0,4253,CNTSN,2022-11-17 09:00:00,127.0609,34.1211,16.1,267.0,2022-11-16 17:01:11.033821,2022-11-16 08:00:55,2022-11-16 04:19:07,under way using engine,186.658612,312.035073
2,COSCO FOS,477947700,9484302.0,4253,CNTSN,2022-11-17 09:00:00,126.6711,34.0831,16.1,262.0,2022-11-16 18:16:53.451054,2022-11-16 09:14:25,2022-11-16 04:19:07,under way using engine,220.108466,367.952814
3,COSCO FOS,477947700,9484302.0,4253,CNTSN,2022-11-17 09:00:00,126.6067,34.0795,16.0,277.0,2022-11-16 18:26:58.801742,2022-11-16 09:26:25,2022-11-16 04:19:07,under way using engine,225.577733,374.509054
4,COSCO FOS,477947700,9484302.0,4253,CNTSN,2022-11-17 09:00:00,126.5502,34.0820,16.2,280.0,2022-11-16 18:42:07.917210,2022-11-16 09:37:09,2022-11-16 04:19:07,under way using engine,230.117608,387.353392
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-21 08:00:00,128.8286,35.0767,0.9,80.0,2022-11-21 18:05:52.289081,2022-11-21 08:45:01,2022-11-21 08:37:08,under way using engine,11.938104,105.884565
529,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-21 08:00:00,128.8303,35.0770,0.1,91.0,2022-11-21 20:12:03.039219,2022-11-21 10:46:03,2022-11-21 10:49:07,moored,11.983151,956.161110
530,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-21 08:00:00,128.8302,35.0772,0.0,91.0,2022-11-21 20:17:05.890117,2022-11-21 10:52:05,2022-11-21 10:55:06,moored,12.004568,NaN
531,COSCO FOS,477947700,9484302.0,4253,KRBNP,2022-11-21 08:00:00,128.8300,35.0772,0.0,91.0,2022-11-21 20:42:20.130436,2022-11-21 11:15:59,2022-11-21 11:13:06,moored,12.003097,NaN


In [142]:
cosco_df["timestamp"] = pd.to_datetime(cosco_df["timestamp"])
cosco_df["eta"] = pd.to_datetime(cosco_df["eta"])
cosco_df["time_position_data_received"] = pd.to_datetime(cosco_df["time_position_data_received"])
cosco_df["time_voyage_data_received"] = pd.to_datetime(cosco_df["time_voyage_data_received"])


In [143]:
cosco_df = cosco_df[cosco_df["timestamp"] > datetime.datetime(year=2022, month=11, day=19, hour=22, minute=24)]
cosco_df.reset_index(inplace=True, drop=True)

In [144]:
cosco_df["현재위치차"] = cosco_df["timestamp"] - cosco_df['time_position_data_received']

In [145]:
for idx, teu in enumerate(cosco_df["CAPACITY"].values):
    mile = float(cosco_df.loc[idx, "remained_distance"]) / 1.852
    S = cosco_df.loc[idx, 'speed']
    T = mile / S
    estimated_fuel_consumption = fuel_estimation(teu, T, S)

In [146]:
cosco_df["predicted_ETA"] = ''
cal_idx = cosco_df.query(
    'navigation_info == "under way using engine"').index
remained_distance = cosco_df.loc[cal_idx, 'remained_distance']
speed_series = cosco_df.loc[cal_idx, "speed"] * 1.852
time_position_data_received = cosco_df.loc[
    cal_idx, 'timestamp']
for idx, remained_distance in zip(remained_distance.index,
                                  remained_distance.values):
    speed = speed_series[idx]
    if speed != 0.0:
        hour = int(remained_distance // speed)
        minute = int(
            (remained_distance / speed - remained_distance // speed) * 60)
        position_time = time_position_data_received[idx]
        predicted_eta = position_time + datetime.timedelta(hours=hour,
                                                           minutes=minute)
        cosco_df.loc[idx, 'predicted_ETA'] = predicted_eta
    else:
        false = datetime.datetime.strptime("1970-01-01 11:59:59", "%Y-%m-%d %H:%M:%S")
        cosco_df.loc[idx, 'predicted_ETA'] = false

In [148]:
cosco_df.to_csv("cosco_df.csv", encoding="utf-8 sig", header=True, index=False)

In [153]:
dddf =cosco_df[["vessel_name","eta", "speed", "timestamp", "remained_distance", "estimated_fuel_consumption", "predicted_ETA"]].drop(304)
dddf

,vessel_name,eta,speed,timestamp,remained_distance,estimated_fuel_consumption,predicted_ETA
0,COSCO FOS,2022-11-21 08:00:00,11.5,2022-11-19 22:24:43.599696,1071.303386,1386.146465,2022-11-22 00:42:43.599696
1,COSCO FOS,2022-11-21 08:00:00,11.6,2022-11-19 22:29:46.266674,1069.727209,1388.920303,2022-11-22 00:16:46.266674
2,COSCO FOS,2022-11-21 08:00:00,11.5,2022-11-19 22:39:53.134121,1065.901997,1379.157672,2022-11-22 00:41:53.134121
3,COSCO FOS,2022-11-21 08:00:00,11.6,2022-11-19 22:49:57.626655,1062.327066,1379.312051,2022-11-22 00:15:57.626655
4,COSCO FOS,2022-11-21 08:00:00,11.6,2022-11-19 22:55:01.041397,1060.642543,1377.124888,2022-11-22 00:17:01.041397
...,...,...,...,...,...,...,...
303,COSCO FOS,2022-11-21 08:00:00,3.9,2022-11-21 17:55:45.279861,11.610739,24.556266,2022-11-21 19:31:45.279861
305,COSCO FOS,2022-11-21 08:00:00,0.1,2022-11-21 20:12:03.039219,11.983151,956.161110,
306,COSCO FOS,2022-11-21 08:00:00,0.0,2022-11-21 20:17:05.890117,12.004568,NaN,
307,COSCO FOS,2022-11-21 08:00:00,0.0,2022-11-21 20:42:20.130436,12.003097,NaN,


In [159]:
dddf.rename({"vessel_name" : "선명"})

,vessel_name,eta,speed,timestamp,remained_distance,estimated_fuel_consumption,predicted_ETA
0,COSCO FOS,2022-11-21 08:00:00,11.5,2022-11-19 22:24:43.599696,1071.303386,1386.146465,2022-11-22 00:42:43.599696
1,COSCO FOS,2022-11-21 08:00:00,11.6,2022-11-19 22:29:46.266674,1069.727209,1388.920303,2022-11-22 00:16:46.266674
2,COSCO FOS,2022-11-21 08:00:00,11.5,2022-11-19 22:39:53.134121,1065.901997,1379.157672,2022-11-22 00:41:53.134121
3,COSCO FOS,2022-11-21 08:00:00,11.6,2022-11-19 22:49:57.626655,1062.327066,1379.312051,2022-11-22 00:15:57.626655
4,COSCO FOS,2022-11-21 08:00:00,11.6,2022-11-19 22:55:01.041397,1060.642543,1377.124888,2022-11-22 00:17:01.041397
...,...,...,...,...,...,...,...
303,COSCO FOS,2022-11-21 08:00:00,3.9,2022-11-21 17:55:45.279861,11.610739,24.556266,2022-11-21 19:31:45.279861
305,COSCO FOS,2022-11-21 08:00:00,0.1,2022-11-21 20:12:03.039219,11.983151,956.161110,
306,COSCO FOS,2022-11-21 08:00:00,0.0,2022-11-21 20:17:05.890117,12.004568,NaN,
307,COSCO FOS,2022-11-21 08:00:00,0.0,2022-11-21 20:42:20.130436,12.003097,NaN,


In [161]:
dddf.rename(columns={"vessel_name" : "선명", "eta":"ais_ETA", "speed" : "선속","timestamp":"현재시각","remained_distance":"남은 거리(km)","estimated_fuel_consumption":"예상 연료 소모량","predicted_ETA":"예측 ETA"})

,선명,ais_ETA,선속,현재시각,남은 거리(km),예상 연료 소모량,예측 ETA
0,COSCO FOS,2022-11-21 08:00:00,11.5,2022-11-19 22:24:43.599696,1071.303386,1386.146465,2022-11-22 00:42:43.599696
1,COSCO FOS,2022-11-21 08:00:00,11.6,2022-11-19 22:29:46.266674,1069.727209,1388.920303,2022-11-22 00:16:46.266674
2,COSCO FOS,2022-11-21 08:00:00,11.5,2022-11-19 22:39:53.134121,1065.901997,1379.157672,2022-11-22 00:41:53.134121
3,COSCO FOS,2022-11-21 08:00:00,11.6,2022-11-19 22:49:57.626655,1062.327066,1379.312051,2022-11-22 00:15:57.626655
4,COSCO FOS,2022-11-21 08:00:00,11.6,2022-11-19 22:55:01.041397,1060.642543,1377.124888,2022-11-22 00:17:01.041397
...,...,...,...,...,...,...,...
303,COSCO FOS,2022-11-21 08:00:00,3.9,2022-11-21 17:55:45.279861,11.610739,24.556266,2022-11-21 19:31:45.279861
305,COSCO FOS,2022-11-21 08:00:00,0.1,2022-11-21 20:12:03.039219,11.983151,956.161110,
306,COSCO FOS,2022-11-21 08:00:00,0.0,2022-11-21 20:17:05.890117,12.004568,NaN,
307,COSCO FOS,2022-11-21 08:00:00,0.0,2022-11-21 20:42:20.130436,12.003097,NaN,
